# Lab 3: Finetune Llama 3.2 on Medical Dataset

This notebook implements finetuning of Llama 3.2 on a medical dataset using Hugging Face Transformers and PEFT (Parameter-Efficient Fine-Tuning).



## Step 1: Install Required Libraries

In [ ]:

%pip install -q transformers datasets peft accelerate bitsandbytes trl torch

## Step 2: Import Libraries and Set Up


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import json
import os
from datetime import datetime
import re

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"✅ Using device: {device} (Apple Silicon)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ Using device: {device}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    device = torch.device("cpu")
    print(f"⚠️ Using device: {device} (CPU - training will be slower)")
    print("Note: If you have a GPU, make sure CUDA is properly installed.")

## Step 3: Load and Prepare Dataset

We'll use the MedMCQA dataset for medical question answering.

In [ ]:
# Load the MedMCQA dataset
dataset = load_dataset("openlifescienceai/medmcqa", split="train")

print(f"Dataset size: {len(dataset)}")
print(f"Dataset features: {dataset.features}")
print("\nSample example:")
print(dataset[0])


## Step 4: Format Dataset for Instruction Tuning

We need to format the dataset in a way that the model can learn from. We'll create a prompt template.

In [ ]:
def format_instruction(example):
    """Format medical Q&A into instruction format"""
    question = example.get("question", "")
    # Extract options from 'opa', 'opb', 'opc', 'opd'
    options = {}
    if 'opa' in example and example['opa'] is not None: options['a'] = example['opa']
    if 'opb' in example and example['opb'] is not None: options['b'] = example['opb']
    if 'opc' in example and example['opc'] is not None: options['c'] = example['opc']
    if 'opd' in example and example['opd'] is not None: options['d'] = example['opd']

    cop_idx = example.get("cop")  # Correct option index (0 for 'a', 1 for 'b', etc.)

    # Build options list
    # Ensure options are sorted by key (a, b, c, d)
    sorted_option_keys = sorted(options.keys())
    options_text = "\n".join([f"{k.upper()}. {options[k]}" for k in sorted_option_keys])

    # Get correct answer text
    correct_answer_text = ""
    if cop_idx is not None and 0 <= cop_idx < len(sorted_option_keys):
        correct_answer_key = sorted_option_keys[cop_idx] # 'a', 'b', 'c', or 'd'
        correct_answer_text = options[correct_answer_key]

    # Create prompt
    prompt = f"""You are a medical expert. Answer the following medical question.

Question: {question}

Options:
{options_text}

Answer: {correct_answer_text}"""

    # Return 'text' for training, and also original question, options, and cop for later evaluation
    return {"text": prompt, "original_question": question, "original_options": options, "original_cop": cop_idx}

# Format dataset (using subset for faster training)
dataset_size = min(2000, len(dataset))
dataset_subset_for_formatting = dataset.select(range(dataset_size)) # Explicitly define the subset used for formatting

formatted_dataset = dataset_subset_for_formatting.map(
    format_instruction,
    # Temporarily remove remove_columns to ensure all new columns are added
)

print(f"Formatted dataset features: {formatted_dataset.features}")

## Local Inference on GPU
Model page: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Model configuration
model_name = "meta-llama/Llama-3.2-1B-Instruct"


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model
print("\nLoading model...")
# MPS doesn't support float16 well, use float32. CUDA can use float16 for efficiency
if torch.backends.mps.is_available():
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,
        device_map=None,  # MPS doesn't support device_map="auto"
        trust_remote_code=True
    )
    model = model.to(device)  # Manually move to MPS device
elif torch.cuda.is_available():
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,
        device_map=None,
        trust_remote_code=True
    )
    model = model.to(device)

print(f"Model loaded: {model_name}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e9:.2f}B")

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you.
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [ ]:
import os
from google.colab import userdata

# Retrieve the HF_TOKEN from Colab secrets
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

print(completion.choices[0].message)

## Step 6: Configure LoRA for Parameter-Efficient Fine-Tuning

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # Rank
    lora_alpha=32,  # LoRA alpha
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Enable input gradients for gradient checkpointing with PEFT
# This is crucial when using gradient_checkpointing with PEFT models
# to ensure the backward pass works correctly through frozen layers.
if training_args.gradient_checkpointing:
    model.enable_input_require_grads()

model.print_trainable_parameters()

print("\nLoRA configuration applied successfully!")

## Step 7: Tokenize Dataset


In [ ]:
def tokenize_function(examples):
    """Tokenize the text examples"""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length",
    )

# Define columns to remove after formatting and tokenization
# Keep 'input_ids', 'attention_mask', 'original_question', 'original_options', 'original_cop'
columns_to_keep = ['input_ids', 'attention_mask', 'original_question', 'original_options', 'original_cop']

# Get all columns in the formatted_dataset
all_formatted_columns = formatted_dataset.column_names

# Identify columns to remove: all columns except those we want to keep
columns_to_remove_after_tokenization = [col for col in all_formatted_columns if col not in columns_to_keep]

# Tokenize dataset
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_after_tokenization,
)

print(f"Tokenized dataset size: {len(tokenized_dataset)}")
print(f"Sample tokenized example keys: {tokenized_dataset[0].keys()}")
print(f"Tokenized dataset features: {tokenized_dataset.features}")

## Step 8: Split Dataset into Train/Test

In [ ]:
# Split dataset: 80% train, 20% test
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Training examples: {len(train_dataset)}")
print(f"Test examples: {len(test_dataset)}")

## Step 9: Set Up Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama-medical-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Reduced from 4 to 1 for MPS memory
    per_device_eval_batch_size=1,  # Reduced from 4 to 1 for MPS memory
    gradient_accumulation_steps=16,  # Increased from 4 to 16 to maintain effective batch size (1*16=16)
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),  # Only enable for CUDA, not MPS
    gradient_checkpointing=True,  # Enable to save memory
    logging_steps=10,
    save_steps=500,
    eval_strategy="steps",  # Changed from evaluation_strategy to eval_strategy
    eval_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="none",
    push_to_hub=False,
)

print("Training arguments configured!")

## Step 10: Create Data Collator

In [ ]:
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing causal LM, not masked LM
)

print("Data collator created!")


## Step 11: Initialize Trainer and Start Training

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

print("Trainer initialized. Starting training...")
print("This may take a while depending on your hardware...")


In [ ]:
# Start training
trainer.train()

## Step 12: Save the Fine-Tuned Model

In [ ]:
# Save the model
model.save_pretrained("./llama-medical-finetuned")
tokenizer.save_pretrained("./llama-medical-finetuned")

print("Model saved successfully!")


## Step 13 : Evaluation, loading test examples

In [ ]:
# load test examples from the split tokenized_dataset
test_examples = []
for i in range(len(test_dataset)):
    example = test_dataset[i]
    test_examples.append({
        "question": example["original_question"],
        "options": example["original_options"],
        "cop": example["original_cop"]
    })

print(f"Loaded {len(test_examples)} test examples")
print("\nSample test example:")
print(test_examples[0])


## Step 14: Create Inference Function


In [ ]:
def generate_answer(question, options, model, tokenizer, max_length=512):
    """Generate an answer for a medical question"""
    #format the prompt
    options_text = ""
    if options:
        for key in sorted(options.keys()):
            option_label = key.upper()
            option_text = options[key]
            options_text += f"{option_label}. {option_text}\n"

    prompt = f""" You are a medical expert.Answer the following medical question.

Question: {question}
Options:
{options_text.strip()}
Answer:"""

    # tokenize
    inputs =tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    inputs= {k: v.to(model.device) for k, v in inputs.items()}

    # generate
    with torch.no_grad() :
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7 ,
            do_sample=True ,
            pad_token_id=tokenizer.eos_token_id,
        )

    # decode
    generated_text =tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer =generated_text.split("Answer:")[-1].strip()

    return answer

# test the function
sample_question =test_examples[0]["question"]
sample_options= test_examples[0]["options"]
sample_answer =generate_answer(sample_question, sample_options, model, tokenizer)
print("Sample generation :")
print(f"Question:{sample_question}")
print(f"Generated Answer:{sample_answer}")

## Step 15: Implement Accuracy Checking


In [ ]:
def check_answer_match(prediction,ground_truth, options):
    """Check if prediction matches ground truth (exact or partial)"""
    # get ground truth text
    if options and isinstance(ground_truth, int):
        option_keys = sorted(options.keys())
        if 0 <= ground_truth < len(option_keys):
            ground_truth_text = options[option_keys[ground_truth]]
        else:
            return False, "no_match"
    else:
        ground_truth_text = str(ground_truth)

    #normalize texts
    prediction_lower = prediction.lower().strip()
    ground_truth_lower = ground_truth_text.lower().strip()
    #check exact match
    if ground_truth_lower in prediction_lower or prediction_lower in ground_truth_lower:
        return True, "exact_match"

    #check keywords
    ground_truth_words = set(ground_truth_lower.split())
    prediction_words = set(prediction_lower.split())
    common_words = ground_truth_words.intersection(prediction_words)

    if len(common_words) >= len(ground_truth_words) * 0.5:
        return True, "partial_match"
    return False, "no_match"

#test the function
sample_cop = test_examples[0].get("cop")
if sample_cop is not None:
    match, match_type =check_answer_match(sample_answer, sample_cop, sample_options)
    print(f"Match: {match},Type:{match_type}")

## Step 16: Run Evaluation Loop

In [ ]:
import time
from tqdm import tqdm

#evaluate on a subset of test examples
eval_size = min(20,len(test_examples))
eval_examples =test_examples[:eval_size]

results = {
    "exact_matches": 0 ,
    "partial_matches": 0,
    "incorrect": 0,
    "details": []
}


start_time = time.time()

for i, example in enumerate(tqdm(eval_examples,desc="Evaluating")):
    question = example.get("question", "")
    options = example.get("options", {})
    correct_answer_idx = example.get("cop", None)

    #generate answer
    prediction = generate_answer(question, options, model, tokenizer)

    #check match
    if correct_answer_idx is not None :
        is_match, match_type = check_answer_match(prediction, correct_answer_idx, options)

        # Get ground truth text
        option_keys = sorted(options.keys())
        if 0 <= correct_answer_idx < len(option_keys):
            ground_truth_text = options[option_keys[correct_answer_idx]]
        else:
            ground_truth_text = f"Option {correct_answer_idx}"

        if match_type == "exact_match":
            results["exact_matches"] += 1
            status = "CORRECT"
        elif match_type == "partial_match":
            results["partial_matches"] += 1
            status = "PARTIAL"
        else:
            results["incorrect"] += 1
            status = "INCORRECT "

        results["details"].append({
            "question": question[:100] + "..." if len(question) > 100 else question,
            "ground_truth": ground_truth_text,
            "prediction": prediction[:200] + "..." if len(prediction) > 200 else prediction,
            "match_type": match_type,
            "correct": is_match
        })

        print(f"\n{status}")
        print(f"Question: {question[:150]}...")
        print(f"Ground Truth: {ground_truth_text}")
        print(f"Prediction: {prediction[:150]}...")
        print("-" * 80)

total_time = time.time()- start_time
results["total_time"] = total_time
results["avg_time_per_example"] = total_time / eval_size

print(f"\nRunning accuracy: {(results['exact_matches'] +results['partial_matches']) / eval_size * 100:.1f}% ({results['exact_matches'] + results['partial_matches']}/{eval_size})")

## Step 17: Calculate Final Metrics

In [ ]:
total_correct = results["exact_matches"] + results["partial_matches"]
total_examples =eval_size
exact_match_rate =results["exact_matches"] / total_examples * 100
partial_match_rate = results["partial_matches"] / total_examples * 100
overall_accuracy = total_correct / total_examples * 100


print("FINAL RESULTS")
print(f"\nTotal examples evaluated: {total_examples}")
print(f"Exact matches: {results['exact_matches']} ({exact_match_rate:.1f}%)")
print(f"Partial matches: {results['partial_matches']} ({partial_match_rate:.1f}%)")
print(f"Total correct: {total_correct} ({overall_accuracy:.1f}%)")
print(f"Incorrect: {results['incorrect']} ({100 - overall_accuracy:.1f}%)")
print(f"\nTotal evaluation time: {total_time / 60:.1f} minutes")
print(f"Average time per example: {results['avg_time_per_example']:.1f} seconds")

## Step 18: Analyze Detailed Results

In [ ]:

print("DETAILED RESULTS")


#separate correct and incorrect examples
correct_examples = [r for r in results["details"] if r["correct"]]
incorrect_examples = [r for r in results["details"] if not r["correct"]]

print(f"\nINCORRECT EXAMPLES ({len(incorrect_examples)}):")

for i, ex in enumerate(incorrect_examples[:10], 1):
    print(f"\n{i}. Question: {ex['question']}")
    print(f"   Ground Truth: {ex['ground_truth']}")
    print(f"   Prediction: {ex['prediction']}")

if len(correct_examples) > 0:
    print(f"\nCORRECT EXAMPLES ({len(correct_examples)}):")

    for i, ex in enumerate(correct_examples[:5], 1):
        print(f"\n{i}. Question: {ex['question']}")
        print(f"   Ground Truth: {ex['ground_truth']}")
        print(f"   Prediction: {ex['prediction']}")
        print(f"   Match type: {ex['match_type']}")

## Step 19: Assess Performance

In [ ]:

print("PERFORMANCE ASSESSMENT")


if overall_accuracy >= 80:
    assessment = "EXCELLENT. Model performs very well."
    recommendation = "Model is ready for further testing and potential deployment."
elif overall_accuracy >= 60:
    assessment = "GOOD. Model shows promise."
    recommendation = "Consider more training epochs or data augmentation."
elif overall_accuracy >= 40:
    assessment = "MODERATE. Model needs improvement."
    recommendation = "Increase training data, adjust hyperparameters, or try different architectures."
else:
    assessment = "VERY POOR. Model barely learned."
    recommendation = "Verify data formatting and retrain from scratch."

print(f"\n{assessment}")
print(f"Recommendation: {recommendation}")

## Step 20: Save Results

In [ ]:
# saving results in a json file
results_summary = {
    "total_examples": total_examples,
    "exact_matches": results["exact_matches"],
    "partial_matches": results["partial_matches"],
    "incorrect": results["incorrect"],
    "exact_match_rate": exact_match_rate,
    "partial_match_rate": partial_match_rate,
    "overall_accuracy": overall_accuracy,
    "evaluation_time": total_time,
    "avg_time_per_example": results["avg_time_per_example"],
    "timestamp": datetime.now().isoformat(),
    "details": results["details"]
}

with open("evaluation_results.json", "w") as f:
    json.dump(results_summary, f, indent=2)


## Part A: Model Improvement Strategies

### Question 1: Improving Model Performance

Based on your evaluation results, propose at least 2 or 3 specific strategies to improve your model's accuracy.

1. Increase training dataset size from 2000 to 5000/10000 for more diverse medical scenarios

2. adjust Lora parameters, as increasing rank to 32 or 64 for the model to learn more complex patterns


### Question 2: Analyzing Failure Patterns

Review your incorrect predictions and identify patterns in failures.

In [ ]:

print("FAILURE PATTERN ANALYSIS")

failure_analysis = {
    "too_verbose": [],
    "wrong_concept": [],
    "partial_understanding": [],
    "hallucination": []
}

for ex in incorrect_examples:
    pred = ex["prediction"].lower()
    gt = ex["ground_truth"].lower()

    # verbose answers
    if len(ex["prediction"]) > len(ex["ground_truth"]) * 3:
        failure_analysis["too_verbose"].append(ex)
    # different medical words
    elif not any(word in pred for word in gt.split()[:3]):
        failure_analysis["wrong_concept"].append(ex)
    # partial understanding
    elif any(word in pred for word in gt.split()):
        failure_analysis["partial_understanding"].append(ex)
    else:
        failure_analysis["hallucination"].append(ex)
print(f"verbose answers: {len(failure_analysis['too_verbose'])}")
print(f"different medical words: {len(failure_analysis['wrong_concept'])}")
print(f"partial understanding: {len(failure_analysis['partial_understanding'])}")
print(f"hallucination {len(failure_analysis['hallucination'])}")

### Question 3: Data Quality vs. Quantity

What do you think is better between training on 2000 examples (same quality) or 500 curated high-quality examples?

***500 curated high-quality examples as low quality examples can teach the model incorrect patterns, and 500 is still diverse with clean data***


## Part B: Resource-Constrained Inference

### Question 4: Optimizing for Limited Resources

How can you design a strategy to reduce inference time/memory for deployment in constrained environments?


- Use 8bit or 4bit to reduce memory
- Model pruning
- Switch from 3B to 1B or even smaller variants

### Question 5: Speed vs. Accuracy Trade-offs

Analyze how changing generation parameters affects speed, quality, and consistency.

In [ ]:
# test different generation parameters
import time
test_question = test_examples[0]["question"]
test_options = test_examples[0]["options"]

configs = [
    {"max_new_tokens": 50, "temperature": 0.1, "do_sample": False, "name": "Greedy, Short"},
    {"max_new_tokens": 100, "temperature": 0.7,"do_sample": True, "name": "Sampling, Medium"},
    {"max_new_tokens": 200, "temperature": 1.0,"do_sample": True, "name": "Sampling, Long, High Temp"},
]

print("SPEED vs ACCURACY ANALYSIS")

for config in configs:
    start = time.time()
    # generate with different configs
    inputs = tokenizer(f"Question: {test_question}\nAnswer:", return_tensors="pt")
    inputs ={k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=config["max_new_tokens"],
            temperature=config["temperature"] ,
            do_sample=config["do_sample"] ,
            pad_token_id=tokenizer.eos_token_id,
        )

    elapsed =time.time() -start
    answer= tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[-1].strip()

    print(f"\n{config['name']}:")
    print(f"Time: {elapsed:.3f}s")
    print(f"Length: {len(answer)} chars")
    print(f"Answer: {answer[:100]}...")



- Lower temperature (0.1-0.5): More focused, consistent answers, faster
- Higher temperature (0.7-1.0): More diverse but potentially less accurate, slower£
- Shorter max_new_tokens: Faster inference, but may truncate answers
- Longer max_new_tokens: Slower, but allows complete answers

## Part C: Evaluation Methodology

### Question 7: Improving Evaluation Metrics

Analyze limitations of current exact/partial match evaluation and propose improvements.

Limitations:

1. False Negatives : Model may give correct answer but in different wording
2. False Positives. : Partial matches might accept incorrect
3. No Semantic Understanding: Doesn't use embeddings to check meaning similarity
4. No Medical Terms Handling: Doesn't account for synonyms

Proposed Improvements:
1. Semantic similarity
2. Medical NER
3. Human evaluation
4. Confidence scores


### Question 8: Test Set Size and Confidence

Test other test sizes and observe the results. What can you say about the results?


In [ ]:

test_sizes = [10, 20, 50, 100]

print("TEST SET SIZE ANALYSIS")


for size in test_sizes:
    if size >len(test_examples):
        continue

    eval_subset =test_examples[:size]
    correct =0

    for example in eval_subset:
        question =example.get("question", "")
        options =example.get("options", {})
        correct_answer_idx = example.get("cop", None)

        prediction =generate_answer(question, options, model, tokenizer)

        if correct_answer_idx is not None:
            is_match, _ = check_answer_match(prediction, correct_answer_idx, options)
            if is_match:
                correct += 1

    accuracy = correct / size * 100
    print(f"Test size: {size:3d} | Accuracy: {accuracy:5.1f}% | Correct: {correct}/{size}")

print("larger test sets provide more reliable estimates")
print("Small test sets have high variance")
print("Need at least 50-100 examples for a accurate result")

## Part D: Real-World Deployment Scenario

### Question 9: Production Considerations

What can you do to address safety, reliability, updates, and edge cases for deploying in a medical assistance application?


1. Safety:
   - Add disclaimers
   - Implement confidence thresholds
   - Input validation

2. Reliability
   - Track accuracy, latency, error rates
   - Redundancy
   - Error handling

3. Updates
   - Track model versions and performance
   - A/B testing between models before starting
   - Quick revert to previous model version

4. Edge Cases
   - Out-of-domain questions
   - Ambiguous questions
   - Handle different languages
   - Context length limits